In [216]:
# Imports

import gspread as gs
import pandas as pd
import plotly.graph_objects as go

In [217]:
# Match Results Google Sheet
sheet_id = "1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8"
sheet_name = "Match_Results_2"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
results_df = pd.read_csv(url).fillna('')
results_df.head(50)

,Timestamp,Match Week,Match Winner,Match Loser,Games Won By Loser,player1,player2,player1score,player2score
0,,,Missy Kayko,Bob Sauchelli,,Missy Kayko,Bob Sauchelli,1,0
1,,,Adam Fratino,Pat Murphy,,Adam Fratino,Pat Murphy,1,0
2,,,Mike Dunlap,Damir Uzunic,,Mike Dunlap,Damir Uzunic,1,0
3,,,Eric Papa,Kofi Wilson,,Eric Papa,Kofi Wilson,1,0
4,,,Kurowska,Amelia Burger,,Kurowska,Amelia Burger,1,0
5,,,Eric Papa,Kurowska,,Eric Papa,Kurowska,1,0
6,,,Missy Kayko,Paul Assad,,Missy Kayko,Paul Assad,1,0
7,,,Mike Dunlap,Eric Papa,,Mike Dunlap,Eric Papa,1,0
8,,,Missy Kayko,Mike Dunlap,,Missy Kayko,Mike Dunlap,1,0
9,,,Justin Goodfellow,Demelo,,Justin Goodfellow,Demelo,1,0


In [218]:
# Match Results Google Sheet
# gc = gs.service_account(filename='shuffleboard-325614-64ddad20cd94.json')
# sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8/edit#gid=1110865845')
# ws = sh.worksheet('Match_Results_2')


In [219]:
# Creating Won_Loss Results Table
winners = dict(enumerate(results_df['Match Winner']))
losers = dict(enumerate(results_df['Match Loser']))
winners.update(losers)
players = winners
wins = results_df['player1'].value_counts()
losses = results_df['player2'].value_counts()
w_l = pd.concat([wins,losses],axis=1).fillna(0).astype(int).round().rename(columns={'player1':'Wins','player2':'Losses'})
w_l.loc[:,'Matches']=w_l.sum(numeric_only=True, axis=1)
w_l['Win%'] = w_l['Wins']/w_l['Matches']
w_l['Player']=w_l.index
column_names = ['Player',"Matches", "Wins", "Losses", "Win%"]
w_l['Win%'] = w_l['Win%'].round(3)
w_l = w_l.reindex(columns=column_names)
w_l = w_l.reset_index(drop=True)
w_l.head(5)

,Player,Matches,Wins,Losses,Win%
0,Mike Dunlap,6,5,1,0.833
1,Ben Brown,5,5,0,1.000
2,Missy Kayko,5,4,1,0.800
3,Eric Papa,4,3,1,0.750
4,Jeff Ziev,5,3,2,0.600


In [220]:
# Rankings
rankings_df = pd.read_csv('Rankings.csv')
rankings_df.drop_duplicates(subset=['Player'],inplace=True)
rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.loc[:,'ELO']=1600
initial_ranking = 1600
k_factor = 32
rankings_df.head(20)

,Player,ELO
Player,,
Ben Brown,Ben Brown,1600
Mike Dunlap,Mike Dunlap,1600
Missy Kayko,Missy Kayko,1600
Eric Papa,Eric Papa,1600
Ryan Leggette,Ryan Leggette,1600
Paul Assad,Paul Assad,1600
Tom Witteman,Tom Witteman,1600
Jeff Weber,Jeff Weber,1600
Jeff Ziev,Jeff Ziev,1600


In [221]:
# ELO Formula
def elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor):
    R1 = 10**(p1_ranking/400)
    R2 = 10**(p2_ranking/400)
    exp_1 = R1/(R1+R2)
    exp_2 = R2/(R1+R2)
    ELO_1 = p1_ranking+k_factor*(p1_score-exp_1)
    ELO_2 = p2_ranking+k_factor*(p2_score-exp_2)
    return ELO_1, ELO_2

In [222]:
#this loop reads the scores in order starting from the top of the matches
#every time you run this loop it will read all of the scores again
for idx in results_df.index:
    player_1 = results_df.player1[idx]
    p1_score = results_df.player1score[idx]
    player_2 = results_df.player2[idx]
    p2_score = results_df.player2score[idx]
    current_rankings_list = list(set(rankings_df.index))
    if player_1 in current_rankings_list:
        p1_ranking = rankings_df.at[player_1,'ELO']
    else:
        p1_ranking = initial_ranking
    if player_2 in current_rankings_list:
        p2_ranking = rankings_df.at[player_2,'ELO']
    else:
        p2_ranking = initial_ranking
    
    ELO_1, ELO_2 = elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor)
    
    if player_1 in current_rankings_list:
        rankings_df.at[player_1,'ELO'] = ELO_1
    else:
        new_ELO_dict = {'Player':player_1,'ELO':ELO_1}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
    if player_2 in current_rankings_list:
        rankings_df.at[player_2,'ELO'] = ELO_2
    else:
        new_ELO_dict = {'Player':player_2,'ELO':ELO_2}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.reset_index(drop=True,inplace=True)
rankings_df.sort_values(by=['ELO'],inplace=True,ascending=False,ignore_index=True)
rankings_df.index+=1
rankings_df.to_csv('Rankings.csv',index=False)

In [223]:
pd.set_option('display.max_rows', None)
rankings_df

,Player,ELO
1,Ben Brown,1678
2,Mike Dunlap,1660
3,Missy Kayko,1642
4,Eric Papa,1631
5,Ryan Leggette,1616
6,Paul Assad,1616
7,Tom Witteman,1616
8,Justin Goodfellow,1615
9,David Chester,1615
10,Jeff Ziev,1615


In [224]:
# Formatting and Printing Final Table
final_table = w_l.merge(rankings_df, how="left", on="Player")
final_table_print = final_table.drop('Matches', axis=1)
final_table_print['Ranking'] = final_table_print[["ELO","Win%"]].apply(tuple,axis=1)\
             .rank(ascending=False, method="min").astype(int)
column_names = ['Ranking','Player', "ELO", "Wins", "Losses", "Win%"]
final_table_print = final_table_print.reindex(columns=column_names)
final_table_print = final_table_print.sort_values(by=['Ranking','Win%'],ascending=[True,False])
final_table_print

,Ranking,Player,ELO,Wins,Losses,Win%
1,1,Ben Brown,1678,5,0,1.000
0,2,Mike Dunlap,1660,5,1,0.833
2,3,Missy Kayko,1642,4,1,0.800
3,4,Eric Papa,1631,3,1,0.750
10,5,Tom Witteman,1616,1,0,1.000
5,6,Ryan Leggette,1616,2,1,0.667
8,6,Paul Assad,1616,2,1,0.667
11,8,David Chester,1615,1,0,1.000
6,9,Justin Goodfellow,1615,2,1,0.667
7,9,Jeff Weber,1615,2,1,0.667


In [230]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=list(column_names),
                fill_color='darkblue',
                font_color='white',
                font_size=16,
                align='center'),
    cells=dict(values=[
                final_table_print.Ranking,
                final_table_print.Player,
                final_table_print.ELO,
                final_table_print.Wins,
                final_table_print.Losses, 
                final_table_print['Win%']
                ],
               fill_color='white',
               font_color='black',
               font_size=14,
               align='center',
               height=30,
               line_color='lightgray',
               format=["1234","","#","#","#",".1%"]
               ),
    columnwidth=20)])

fig.show()
fig.write_html("final_table.html", include_plotlyjs="cdn")

final_table_print.to_csv('Rankings_Table.csv',index=False)
final_table_print.to_excel('Rankings_Table.xlsx',index=False)
final_table_print


,Ranking,Player,ELO,Wins,Losses,Win%
1,1,Ben Brown,1678,5,0,1.000
0,2,Mike Dunlap,1660,5,1,0.833
2,3,Missy Kayko,1642,4,1,0.800
3,4,Eric Papa,1631,3,1,0.750
10,5,Tom Witteman,1616,1,0,1.000
5,6,Ryan Leggette,1616,2,1,0.667
8,6,Paul Assad,1616,2,1,0.667
11,8,David Chester,1615,1,0,1.000
6,9,Justin Goodfellow,1615,2,1,0.667
7,9,Jeff Weber,1615,2,1,0.667
